### Challenge
Que ingredientes tiene un flan de turrón<br>
Como hacer un huevo frito (no debería contestar porque no es parte de las recetas).<br>
Cuantos huevos necesita la receta de flan de turrón?<br>
el flan de turrón tiene como ingrediente la manteca?

In [1]:
# IMPORTING THE LIBRARIES
#import fitz
import os
import re
import requests

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt

from chromadb.api.types import EmbeddingFunction
from dotenv import load_dotenv

from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

from langchain.document_loaders import PyPDFLoader
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
from typing import Literal, Optional, Any

In [2]:
def pdf_to_text(path: str, 
                start_page: int = 1, 
                end_page: Optional[int | None] = None) -> list[str]:
    """
    Converts PDF to plain text.
    Params:
        path (str): Path to the PDF file.
        start_page (int): Page to start getting text from.
        end_page (int): Last page to get text from.
    """
    loader = PyPDFLoader(path)
    pages = loader.load()
    total_pages = len(pages)

    if end_page is None:
        end_page = len(pages)

    text_list = []
    for i in range(start_page-1, end_page):
        text = pages[i].page_content
        text = text.replace('\n', ' ')
        text = re.sub(r'\s+', ' ', text)
        text_list.append(text)

    return text_list

In [3]:
# PDF files available:
#    "pdfs/pie_recipe.pdf"
#    "pdfs/paper_flowers.pdf"
text_list = pdf_to_text("data/40-recetas-para-los-men-s-de-navidad.pdf")
print(text_list)

['www.cocina-familiar.com 40 recetas para los menús de Navidad', 'Índice del recetario 1. Flan de turrón 2. Mantecados de pueblo, receta de la tía Reme 3. Canelones de jamón con crema de setas 4. Ensalada de aguacate y gambas para las cenas de Navidad 5. Polvorones caseros de almendra 6. Como cocer gambas y otros mariscos en casa 7. Turrón de chocolate con almendras 8. Turrón de Jijona casero. El turrón blando de toda la vida. 9. Roscón de Reyes casero y esponjoso 10. Cordero asado al horno con patatas para Navidad 11. Como hacer redondo de ternera asado al horno 12. Merluza en salsa verde 13. Sopa de pescado y marisco para Navidad 14. Macedonia de frutas, una saludable receta para Navidad 15. Cóctel de gambas, receta para Navidad 16. Cochinillo asado, receta de mi tía Luisa 17. Pavo relleno, especial para Navidad 18. Turrón de yema tostada, receta para Navidad 19. Canapés fáciles para una fiesta 20. Pastel de merluza y langostinos 21. Canapés o pasabocas, contraste de sabores', '22. L

In [4]:
def text_to_chunks(texts: list[str], 
                   word_length: int = 150, 
                   start_page: int = 1) -> list[list[str]]:
    """
    Splits the text into equally distributed chunks.
    Args:
        texts (str): List of texts to be converted into chunks.
        word_length (int): Maximum number of words in each chunk.
        start_page (int): Starting page number for the chunks.
    """
    text_toks = [t.split(' ') for t in texts]
    chunks = []

    for idx, words in enumerate(text_toks):
        for i in range(0, len(words), word_length):
            chunk = words[i:i+word_length]
            if (i+word_length) > len(words) and (len(chunk) < word_length) and (
                len(text_toks) != (idx+1)):
                text_toks[idx+1] = chunk + text_toks[idx+1]
                continue
            chunk = ' '.join(chunk).strip() 
            chunk = f'[Page no. {idx+start_page}]' + ' ' + '"' + chunk + '"'
            chunks.append(chunk)
            
    return chunks

In [5]:
chunks = text_to_chunks(text_list)
for chunk in chunks:
    print(chunk + '\n')

[Page no. 2] "www.cocina-familiar.com 40 recetas para los menús de Navidad Índice del recetario 1. Flan de turrón 2. Mantecados de pueblo, receta de la tía Reme 3. Canelones de jamón con crema de setas 4. Ensalada de aguacate y gambas para las cenas de Navidad 5. Polvorones caseros de almendra 6. Como cocer gambas y otros mariscos en casa 7. Turrón de chocolate con almendras 8. Turrón de Jijona casero. El turrón blando de toda la vida. 9. Roscón de Reyes casero y esponjoso 10. Cordero asado al horno con patatas para Navidad 11. Como hacer redondo de ternera asado al horno 12. Merluza en salsa verde 13. Sopa de pescado y marisco para Navidad 14. Macedonia de frutas, una saludable receta para Navidad 15. Cóctel de gambas, receta para Navidad 16. Cochinillo asado, receta de mi tía Luisa 17. Pavo relleno, especial para Navidad 18. Turrón de yema tostada, receta para"

[Page no. 3] "Navidad 19. Canapés fáciles para una fiesta 20. Pastel de merluza y langostinos 21. Canapés o pasabocas, cont

In [6]:
%%time
# Load the model from TF Hub
class MiniLML6V2EmbeddingFunction(EmbeddingFunction):
    MODEL = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
    def __call__(self, texts):
        return MiniLML6V2EmbeddingFunction.MODEL.encode(texts).tolist()
emb_function = MiniLML6V2EmbeddingFunction()

CPU times: total: 8.05 s
Wall time: 13.9 s


In [7]:
def get_text_embedding(texts: list[list[str]], 
                       batch: int = 1000) -> list[Any]:
        """
        Get the embeddings from the text.
        Args:
            texts (list(str)): List of chucks of text.
            batch (int): Batch size.
        """
        embeddings = []
        for i in range(0, len(texts), batch):
            text_batch = texts[i:(i+batch)]
            # Embeddings model
            emb_batch = emb_function(text_batch)
            embeddings.append(emb_batch)
        embeddings = np.vstack(embeddings)
        return embeddings

In [8]:
embeddings = get_text_embedding(chunks)
print(embeddings.shape)
print(f"Our text was embedded into {embeddings.shape[1]} dimensions")

(94, 768)
Our text was embedded into 768 dimensions


In [9]:
print(embeddings[0])

[-7.35802427e-02  4.13274653e-02 -7.23147765e-03  6.00956678e-02
  8.32382962e-02  1.45892009e-01  7.43446220e-03  1.13130674e-01
  1.31595224e-01  8.25519413e-02 -6.13138825e-02  1.67302676e-02
  9.42279175e-02  1.09271243e-01 -1.85594391e-02 -9.85678211e-02
  2.88704727e-02  4.57063355e-02 -6.66593090e-02  7.98511729e-02
 -2.61458755e-02  8.81410670e-03  3.16802561e-02 -1.24652602e-01
 -8.63705948e-02 -5.27577214e-02 -2.46376470e-02 -3.26593071e-02
  3.98507155e-02 -7.80659616e-02  1.49082631e-01 -1.45665347e-03
  2.06689872e-02 -5.36327139e-02  3.79024446e-02  2.20856778e-02
 -4.52850126e-02 -3.96358855e-02 -2.46421769e-02  1.23506755e-01
  5.21248505e-02 -1.40576333e-01  1.39023727e-02  1.00349918e-01
 -1.43475205e-01  8.23259205e-02 -6.80971369e-02  3.67549341e-03
  2.10505705e-02  8.80268216e-02  2.56796908e-02 -4.56014983e-02
 -1.84910178e-01 -3.68061177e-02  2.29563072e-01 -6.66645616e-02
 -8.75060782e-02 -2.75662318e-02  7.90096261e-03 -3.11782099e-02
  3.03510875e-02 -3.61311

In [10]:
def visualize_embeddings(embeddings_2d: np.ndarray, 
                         question: Optional[bool] = False, 
                         neighbors: Optional[np.ndarray] = None) -> None:
    """
    Visualize 384-dimensional embeddings in 2D using t-SNE, label each data point with its index,
    and optionally plot a question data point as a red dot with the label 'q'.
    Args:
        embeddings (numpy.array): An array of shape (num_samples, 384) containing the embeddings.
        question (numpy.array, optional): An additional 384-dimensional embedding for the question.
                                          Default is None.
    """
    # Scatter plot the 2D embeddings and label each data point with its index
    plt.figure(figsize=(10, 8))
    num_samples = embeddings.shape[0]
    if neighbors is not None:
        for i, (x, y) in enumerate(embeddings_2d[:num_samples]):
            if i in neighbors:
                plt.scatter(x, y, color='purple', alpha=0.7)
                plt.annotate(str(i), xy=(x, y), xytext=(5, 2), textcoords='offset points', color='black')
            else:
                plt.scatter(x, y, color='blue', alpha=0.7)
                plt.annotate(str(i), xy=(x, y), xytext=(5, 2), textcoords='offset points', color='black')
    else:
        for i, (x, y) in enumerate(embeddings_2d[:num_samples]):
            plt.scatter(x, y, color='blue', alpha=0.7)
            plt.annotate(str(i), xy=(x, y), xytext=(5, 2), textcoords='offset points', color='black')
        
    # Plot the question data point if provided
    if question:
        x, y = embeddings_2d[-1]  # Last point corresponds to the question
        plt.scatter(x, y, color='red', label='q')
        plt.annotate('q', xy=(x, y), xytext=(5, 2), textcoords='offset points', color='black')

    plt.title('t-SNE Visualization of 384-dimensional Embeddings')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.show()

In [11]:
def send_to_watsonxai(prompts,
                    model_name="meta-llama/llama-2-70b-chat", #meta-llama/llama-2-70b-chat  #google/flan-ul2
                    decoding_method="greedy",
                    max_new_tokens=400,
                    min_new_tokens=30,
                    temperature=1.0,
                    repetition_penalty=1.0
                    ):
    '''
   helper function for sending prompts and params to Watsonx.ai
    Args:  
        prompts:list list of text prompts
        decoding:str Watsonx.ai parameter "sample" or "greedy"
        max_new_tok:int Watsonx.ai parameter for max new tokens/response returned
        temperature:float Watsonx.ai parameter for temperature (range 0>2)
        repetition_penalty:float Watsonx.ai parameter for repetition penalty (range 1.0 to 2.0)
    Returns: None
        prints response
    '''
    # Instantiate parameters for text generation
    model_params = {
        GenParams.DECODING_METHOD: decoding_method,
        GenParams.MIN_NEW_TOKENS: min_new_tokens,
        GenParams.MAX_NEW_TOKENS: max_new_tokens,
        GenParams.RANDOM_SEED: 42,
        GenParams.TEMPERATURE: temperature,
        GenParams.REPETITION_PENALTY: repetition_penalty,
    }
    # Instantiate a model proxy object to send your requests
    model = Model(
        model_id=model_name,
        params=model_params,
        credentials=creds,
        project_id=project_id)
    for prompt in prompts:
        print(model.generate_text(prompt))

In [12]:
# Config watsonx.ai environment
load_dotenv()
api_key = os.getenv("API_KEY", None)
ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None)
project_id = os.getenv("PROJECT_ID", None)
if api_key is None or ibm_cloud_url is None or project_id is None:
    print("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

In [13]:
questions = ['Qué ingredientes tiene un flan de turrón?','Cómo hacer un huevo frito?','Cuántos huevos necesita la receta de flan de turrón?','El flan de turrón tiene como ingrediente la manteca?']

In [14]:
def build_prompt(question):
    docs = ""
    #prompt = """[INST]<<SYS>>Eres un experto cocinero. Tus respuestas no deben incluir ningún contenido que dañe, sea no ético, racista, tóxico, peligroso o ilegal. 
    #    Asegúrate de que tus respuestas no tengas sesgo social y sean positivas por naturaleza. 
    #    Si la pregunta o el contexto no tienen sentido, o es incoherente, responde "Lo siento no puedo ayudarte". 
    #    Conteste la pregunta usando sólo las recetas suministradas entre las etiquetas <CONTEXTO> y </CONTEXTO>. 
    #    Si no encuentras el contenido de la respuesta en el contexto suministrado (entre las etiquetas <CONTEXTO> y </CONTEXTO>), 
    #    responder solamente: "Lo siento no puedo ayudarte".
    #    Responder siempre en idioma español, sino no responder.
    #   <CONTEXTO>@Documentos</CONTEXTO>
    #   <</SYS>>
    #   Pregunta: @Pregunta
    #   [/INST]"""

    prompt = """[INST]<<SYS>>You are an expert cooker. Your responses should not include any content that is harmful, unethical, racist, toxic, dangerous or illegal.
        Make sure your answers are free of social bias and positive in nature.
        If the question or context doesn't make sense, or is incoherent, respond in Spanish "Lo siento no puedo ayudarte."
        Answer the question using only the recipes supplied between the <CONTEXT> and </CONTEXT> tags.
        If you can't find the content of the response in the supplied context (between the <CONTEXT> and </CONTEXT> tags),
        respond only in Spanish "Lo siento no puedo ayudarte."
        Always respond in Spanish, otherwise do not respond.
        <CONTEXTO>@Documentos</CONTEXTO>
        <</SYS>>
        Pregunta: @Pregunta
        [/INST]"""
    
    for c in topn_chunks:
        docs += c + '\n\n'

    prompt = prompt.replace("@Documentos",docs)
    prompt = prompt.replace("@Pregunta",question)

    return prompt


In [16]:
# Create a t-SNE model
tsne = TSNE(n_components=2, perplexity=20, random_state=42)
for q in questions:
    emb_question = emb_function([q])
    #embeddings_with_question = np.vstack([embeddings, emb_question])
    #embeddings_2d = tsne.fit_transform(embeddings_with_question)
    #nn_2d = NearestNeighbors(n_neighbors=5)
    #nn_2d.fit(embeddings_2d[:-1])
    #neighbors = nn_2d.kneighbors(embeddings_2d[-1].reshape(1, -1), return_distance=False)
    #visualize_embeddings(embeddings_2d, True, neighbors)
    nn = NearestNeighbors(n_neighbors=5)
    nn.fit(embeddings)
    neighbors = nn.kneighbors(emb_question, return_distance=False)
    topn_chunks = [chunks[i] for i in neighbors.tolist()[0]]
    prompt = build_prompt(q)
    print('\n\n\nPregunta: ' + str(q) + '\n')
    send_to_watsonxai(prompts=[prompt], min_new_tokens=1)




Pregunta: Qué ingredientes tiene un flan de turrón?

  ¡Hola! According to the recipe on page 5, a flan de turrón includes the following ingredients:

* 6 huevos gordos
* 100 g of azúcar
* 500 ml of nata líquida (crema de leche in América)
* Caramelo líquido for the mold
* 250 gr. of turrón de Jijona

I hope that helps! If you have any other questions, feel free to ask.



Pregunta: Cómo hacer un huevo frito?

  Lo siento, no puedo ayudarte con esa pregunta. La receta del huevo frito no se encuentra en el contexto proporcionado.



Pregunta: Cuántos huevos necesita la receta de flan de turrón?

  According to the recipe on page 5, the flan de turrón recipe requires 6 eggs.



Pregunta: El flan de turrón tiene como ingrediente la manteca?

  Lo siento, no puedo ayudarte. La receta del flan de turrón que appears en el contexto no incluye manteca como ingrediente. Los ingredientes utilizados en esta receta son huevos, azúcar, nata líquida, caramelo líquido y turrón de Jijona.


In [17]:
## Challenge 2: no responde bien porque parte de como prepararlo no está en los chunks
questions_2 = ['Como hacer un flan de turrón?','Como es la preparación de un flan de turrón?','Quiero hacer un flan de turrón pero no tengo azucar.']
for q in questions_2:
    emb_question = emb_function([q])
    #embeddings_with_question = np.vstack([embeddings, emb_question])
    #embeddings_2d = tsne.fit_transform(embeddings_with_question)
    #nn_2d = NearestNeighbors(n_neighbors=5)
    #nn_2d.fit(embeddings_2d[:-1])
    #neighbors = nn_2d.kneighbors(embeddings_2d[-1].reshape(1, -1), return_distance=False)
    #visualize_embeddings(embeddings_2d, True, neighbors)
    nn = NearestNeighbors(n_neighbors=5)
    nn.fit(embeddings)
    neighbors = nn.kneighbors(emb_question, return_distance=False)
    topn_chunks = [chunks[i] for i in neighbors.tolist()[0]]
    prompt = build_prompt(q)
    print('\n\n\nPregunta: ' + str(q) + '\n')
    send_to_watsonxai(prompts=[prompt], min_new_tokens=1)




Pregunta: Como hacer un flan de turrón?

  Lo siento, no puedo ayudarte con esa pregunta. No se proporciona información suficiente en el contexto para hacer un flan de turrón. Además, la receta del flan de turrón no está incluida en las páginas proporcionadas. Por favor, proporciona más información o una receta completa para que pueda ayudarte.



Pregunta: Como es la preparación de un flan de turrón?

  Lo siento, no puedo ayudarte con esa pregunta. La receta del flan de turrón no está incluida en el contexto proporcionado.



Pregunta: Quiero hacer un flan de turrón pero no tengo azucar.

  Lo siento, no puedo ayudarte con esa pregunta. La receta del flan de turrón requiere azúcar para su preparación, y no hay sustituto adecuado para esta ingrediente. Sin azúcar, no se puede obtener el sabor y la textura característicos del flan de turrón.

Si no tienes azúcar, podrías considerar utilizar una receta diferente que no requiera este ingrediente. Por ejemplo, podrías intentar hacer un